In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
import scipy as sp
from collections import defaultdict
from itertools import product
import gurobipy as gb
from gurobipy import GRB
from tqdm import tqdm
from itertools import chain
from itertools import permutations

repeats = json.load(open('../cleanerdata/repeats.json'))
repeats = {int(k):v for k,v in repeats.items()}

locdem = pd.read_excel('../cleanerdata/locdem.xlsx')

q = locdem['Number of pallets'].to_numpy().astype(float).tolist()
q = [0] + q
q.extend([v['dem'] for v in repeats.values()])

locs = pd.read_excel('../cleanerdata/locations.xlsx')
longs, lats = locs['long'].to_numpy().tolist(), locs['lat'].to_numpy().tolist()
longs.extend([longs[v['map']] for v in repeats.values()])
lats.extend([lats[v['map']] for v in repeats.values()])

distmat = pd.read_json('../cleanerdata/distmat.json').to_numpy()
timemat = pd.read_json('../cleanerdata/timemat.json').to_numpy()
def get(i):
    try:
        i = repeats[i]['map']
    except KeyError:
        pass
    return i

reverse_mapping = defaultdict(list)
for k,v in repeats.items():
    reverse_mapping[v['map']].append(k)
for k in reverse_mapping:
    reverse_mapping[k].append(k)

mappings = list(chain(*[permutations(ls,2) for ls in reverse_mapping.values()]))

def cost(i,j):
    if (i,j) in mappings:
        return 1e+8
    i,j = get(i), get(j)
    return distmat[i,j]

def time(i,j):
    i,j = get(i), get(j)
    return timemat[i,j]

def location(i):
    return longs[i], lats[i]

In [2]:
clusters = json.load(open('./clusters.json'))
list(map(lambda x: len(x), clusters))

[30, 30, 30, 30, 30, 30]

In [ ]:
Q = 9
iter_count = 0
primary_models = []
Ls = []

for cluster in clusters:
    Neighbours = list(sorted(cluster))
    all_nodes = [0] + Neighbours.copy()
    v = len(all_nodes)
    V = list(range(v))
    N = V.copy()
    N.remove(0)
    
    def Del(i):
        F = V.copy()
        F.remove(i)
        return F
    
    m = gb.Model()
    x = m.addVars(v,v, vtype=GRB.BINARY, name='x')
    m.setObjective(sum([cost(all_nodes[i], all_nodes[j]) * x[i,j] for i,j in product(V,V)]))
    for i in N:
        m.addConstr(sum([x[i,j] for j in Del(i)]) == 1)
    for j in N:
        m.addConstr(sum([x[i,j] for i in Del(j)]) == 1)
        
    l = m.addVar(lb=0.0, vtype=GRB.INTEGER, name='L')
    Ls.append(l)
    m.addConstr(sum([x[0,j] for j in Del(0)]) == l)
    
    u = m.addVars(V, vtype=GRB.CONTINUOUS, name='u')
    for i,j in filter(lambda tup: tup[0] != tup[1], product(N,N)):
        m.addConstr(u[i] - u[j] + Q*x[i,j] <= Q - q[j])
    for i in V:
        m.addConstr(u[i] >= q[i])
        
    for pair in mappings:
        if pair[0] in cluster and pair[1] in cluster:
            pair = Neighbours.index(pair[0])+1, Neighbours.index(pair[1])+1
            #print(Neighbours, pair)
            for length in range(1,6):
                all_groups = permutations(N,length)
                for group in all_groups:
                    if pair[0] in group or pair[1] in group:
                        continue
                    group = [pair[0]] + list(group) + [pair[1]]
                    edges = [(group[i], group[i+1]) for i in range(len(group) - 1)]
                    expr = 0
                    for edge in edges:
                        expr += x[edge]
                    m.addConstr(expr <= length)
    m.Params.timelimit=600
    primary_models.append(m)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-12


In [ ]:
for model in tqdm(primary_models):
    model.optimize();

In [ ]:
model_solutions = {}
for i,model in enumerate(primary_models):
    soln = json.loads(model.getJSONSolution())
    model_solutions[i] = soln
json.dump(model_solutions, open('./primary_model_solutions.json', 'w'), indent=4)

In [ ]:
s = 0
for model in primary_models:
    s += model.ObjVal
s

In [ ]:
from itertools import permutations
K = [1,2,3]
print(*permutations(K,2))